## This notebook uses the OPENAI API to summarize a webpage / website. It uses , BeautiulSoup to scrape the data from the website and pass it to the OpenAI API as input to summarize it. The resulting output is displayed nicely using MarkDown

## Import the libraries

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

Load in the environment variables in your `.env` file and connect to OpenAI.

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')


In [3]:
openai = OpenAI()

## Scrape the website and capture the results in the class
##### Beautiful Soup is a library that makes it easy to scrape information from web pages. It sits atop an HTML or XML parser, providing Pythonic idioms for iterating, searching, and modifying the parse tree

In [4]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

## Types of prompts


Models expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [5]:
# Define our system prompt 

system_prompt = "You are an assistant that analyzes the contents of a job posting \
and provides a short summary of the role highlighting salaray expectation and location if available. Ignore text that might be navigation related. \
Summarise the key skillsets required, Technical requirements for the role. \
Respond in markdown."

In [6]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a job titled {website.title}"
    user_prompt += "\nThe details of this role are as follows; \
please provide a short summary of this role in markdown. \
If it includes salary or location details, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]


## Build system and user prompts

In [7]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Summarize the data by calling the open AI API

In [8]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

## Markdown library to summarize the data nicely

In [9]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [11]:
display_summary("https://www.seek.com.au/job/82647026?ref=saved-homepage&pos=1&origin=jobTitle")

# Data Consultant Job Summary

**Location:** Sydney, NSW  
**Salary:** Not explicitly mentioned, but competitive compensation and benefits are highlighted.

## Role Overview
Tranzformd is seeking experienced Senior Data Consultants to join their consultancy team. The role involves collaborating with ASX listed companies to lead and implement data strategies and solutions encompassing data migration, engineering, governance, and analysis.

## Key Responsibilities
- Lead data migration projects and ensure data integrity.
- Design and implement data engineering processes and ETL workflows.
- Develop data governance frameworks for compliance and data quality.
- Conduct data analysis to provide insights for business decisions.
- Offer consultancy services and stakeholder engagement.
- Provide training to clients on data best practices and tools.

## Key Skillsets Required
- **Experience:** Minimum of 4 years in data consultancy, preferably in client-facing roles.
- **Technical Skills:** 
  - Proficiency in data migration tools and ETL processes.
  - Familiarity with cloud platforms (AWS, Azure, Google Cloud).
  - Knowledge of data analysis and visualization tools (SQL, Python, R, Tableau, Power BI).
- **Data Governance:** Strong understanding of data governance and compliance.
- **Soft Skills:** 
  - Excellent problem-solving and analytical skills.
  - Strong communication skills for presenting technical concepts.
  - Ability to manage multiple projects and meet client needs effectively.

## Additional Information
Tranzformd emphasizes a collaborative work environment and supports ongoing professional development and work-life balance.